# PREDICCION DE FASE USANDO NPL

## 1. Preparacion

### 1.1 Importe De Librerias

In [35]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from pylab import *
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell  
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, HTML
from scipy import stats
import matplotlib.pyplot as plt
import datetime
from dateutil.relativedelta import relativedelta
from statistics import mean

import re
import nltk
nltk.download('stopwords')
import pickle
from nltk.corpus import stopwords
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.model_selection import RandomizedSearchCV
import pickle
import nltk
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix

[nltk_data] Downloading package stopwords to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### 1.2 Carga de Base De Datos

In [36]:
path='Data.xlsx'
base = pd.read_excel(path)
data = base

In [37]:
data

,Siglas,Nombre,Desde,Hasta,MD From (ft),MD to (ft),Codigo,Subcodigo,Operación
0,ODR,MOR BL 04,2011-05-25 08:00:00,2011-05-25 11:00:00,2773.0,2773.0,1300,1304.0,"HANG TBG ON 7 1/16"" CASING HEAD. DISCONNECTED ..."
1,ODR,MOR BL 04,2011-05-25 02:30:00,2011-05-25 08:00:00,2773.0,2773.0,1100,1117.0,"RIH 2 7 /8"" TUBING EUE (31.62 FT) + NIPLESILL..."
2,ODR,MOR BL 04,2011-05-25 00:00:00,2011-05-25 02:30:00,2773.0,2773.0,1100,1116.0,CONTD. POOH AND L/D 3 1/2'' DP + SETTING TOOL ...
3,ODR,MOR BL 04,2011-05-24 00:00:00,2011-05-24 03:30:00,2773.0,2773.0,1500,1503.0,"GRAVEL PACKING 5½"" LINER W/ 10-16 MESH GRAVEL ..."
4,ODR,MOR BL 04,2011-05-24 21:30:00,2011-05-24 22:30:00,2773.0,2773.0,1500,1503.0,CONDITIONED BRINE TO 9.5 PPG AND CIRCULATED AN...
...,...,...,...,...,...,...,...,...,...
27180,ODR,MOR D 02,2009-08-22 17:30:00,2009-08-22 19:00:00,2907.0,2907.0,1500,1503.0,CONDITIONED BRINE TO PERFORM CEMENT JOB
27181,ODR,MOR D 02,2009-08-22 10:00:00,2009-08-22 10:30:00,2907.0,2907.0,1700,1701.0,HELD PREOPERATIONAL METTING TO SET PACKER STAG...
27182,ODR,MOR D 02,2009-08-23 00:00:00,2009-08-23 03:30:00,2907.0,2907.0,2100,2104.0,"HELD PREOPERATIONAL MEETING TO 7"" CASING CEMEN..."
27183,ODR,MOR D 02,2009-08-23 03:30:00,2009-08-24 00:00:00,2907.0,2907.0,2100,2106.0,"7""CSG CEMENT WOC"


In [38]:
# Eliminacion de columnas innecesarias y cambio de nombre de columnas
data = data.rename({'Operación':"Operacion"}, axis=1)

## 2. Uso Del Modelo

### 2.1 Cargue Del Modelo

In [39]:
# Cargue del modelo entrenado
model_rf = pickle.load(open("rf.sav", 'rb'))

# Cargue del vectorizador entrenado
vectorizer = pickle.load(open("vectorizer.sav", 'rb'))

# Cargue del codificador de etiquetas
le = pickle.load(open("label_encoder.sav", 'rb'))

### 2.2 Predicción Sobre Un Pozo

#### 2.2.1 Funcion De Seleccion Y Transformacion Para Uso Del Modelo

In [40]:
def pre_pro_well(base,well):
    '''
    FUNCION QUE SELECCIONA UN POZO, TRANSFORMA LOS DATOS COMO INPUT DEL MODELO

    - base: Dataframe con los datos de todos los pozos.
    - well: cadena de caracteres con el nombre del pozo a evaluar    
    '''

    base = base[base["Nombre"]==well]

    X = base["Operación"]

    nltk.download('wordnet')
    documents = []
    
    stemmer = WordNetLemmatizer()

    for sen in range(0, len(X)):

        # Removemos los caracteres especiales
        document = re.sub(r'\W', ' ', str(X[sen]))
        
        # Removemos todos las cadenas de caracteres de un solo caracter
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Removemos caracteres sencillso al inicio
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        
        # Substituimos dobles espacios por espacios sencillos
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removemos caracteres numericos
        document = re.sub(r'\[[0-9]*\]',' ',document)
        
        # Convertimos todo a minusculas
        document = document.lower()
        
        # Realizamos Lematizacion
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
        
        documents.append(document)
    
    return (documents,base)

In [41]:
# Preparamos los datos para la prediccion sobre un pozo especifico
X_val,base_val = pre_pro_well(base,"MOR BL 04")

# Usamos el vectorizador para transformar los datos para la prediccion
X_val = vectorizer.transform(X_val)

[nltk_data] Downloading package wordnet to C:\Users\Diego
[nltk_data]     Ojeda\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#### 2.2.2 Predicción Sobre Los Datos Pre-Procesados

In [46]:
y_val = model_rf.predict(X_val)
base_val["Target"] = le.inverse_transform(y_val)
base_val = base_val[['Siglas', 'Target', 'Nombre', 'Desde', 'Hasta',"MD From (ft)","MD to (ft)","Codigo","Subcodigo","Operación"]];

#### 2.2.3 V

In [45]:
base_val.sort_values(by=['Desde'])[100:]

,Siglas,Target,Nombre,Desde,Hasta,MD From (ft),MD to (ft),Codigo,Subcodigo,Operación
7,ODR,ODR,MOR BL 04,2011-05-24 09:30:00,2011-05-24 13:00:00,2773.0,2773.0,1100,1116.0,"POOH 3 1/2"" DP + GRAVEL PACKER + 2 7/8"" TBG FR..."
13,ODR,ODR,MOR BL 04,2011-05-24 13:00:00,2011-05-24 17:00:00,2773.0,2773.0,1100,1117.0,"M/U FLUTE AND RIH 2 7/8"" TUBING AS STINGER TO ..."
5,ODR,ODR,MOR BL 04,2011-05-24 17:00:00,2011-05-24 18:30:00,2773.0,2773.0,1200,1207.0,SET METAL SEAL W/5000 LBS. RELEASED SETTING TO...
12,ODR,ODR,MOR BL 04,2011-05-24 18:30:00,2011-05-24 20:30:00,2773.0,2773.0,1100,1116.0,POOH WASHING W/ FLUTE 5-1/2'' LINER JOINT TO J...
6,ODR,ODR,MOR BL 04,2011-05-24 20:30:00,2011-05-24 21:30:00,2773.0,2773.0,1100,1116.0,CONTD. POOH AND L/D 3 1/2'' DP + SETTING TOOL ...
4,ODR,ODR,MOR BL 04,2011-05-24 21:30:00,2011-05-24 22:30:00,2773.0,2773.0,1500,1503.0,CONDITIONED BRINE TO 9.5 PPG AND CIRCULATED AN...
11,ODR,ODR,MOR BL 04,2011-05-24 22:30:00,2011-05-25 00:00:00,2773.0,2773.0,1100,1116.0,CONTD. POOH AND L/D 3 1/2'' DP + SETTING TOOL ...
2,ODR,ODR,MOR BL 04,2011-05-25 00:00:00,2011-05-25 02:30:00,2773.0,2773.0,1100,1116.0,CONTD. POOH AND L/D 3 1/2'' DP + SETTING TOOL ...
1,ODR,OCM,MOR BL 04,2011-05-25 02:30:00,2011-05-25 08:00:00,2773.0,2773.0,1100,1117.0,"RIH 2 7 /8"" TUBING EUE (31.62 FT) + NIPLESILL..."
110,OCM,OCM,MOR BL 04,2011-05-25 02:30:00,2011-05-25 08:00:00,2773.0,2773.0,1100,1164.0,"RIH 2 7 /8"" TUBING EUE (31.62 FT) + NIPLESILL..."
